# Method

<div>
<img src="figs/08_retrival_bot.jpg" width="1000"/>
</div>

In [7]:
!wget https://raw.githubusercontent.com/zyds/transformers-code/master/02-NLP%20Tasks/13-retrieval_chatbot/law_faq.csv -P data/

--2023-12-08 15:05:40--  https://raw.githubusercontent.com/zyds/transformers-code/master/02-NLP%20Tasks/13-retrieval_chatbot/law_faq.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18063379 (17M) [text/plain]
Saving to: ‘data/law_faq.csv’

law_faq.csv         100%[===================>]  17.23M  80.1MB/s    in 0.2s    

2023-12-08 15:05:41 (80.1 MB/s) - ‘data/law_faq.csv’ saved [18063379/18063379]



In [24]:
import os
# os.environ["WORLD_SIZE"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
from transformers import DataCollatorWithPadding
import evaluate
from transformers import pipeline
import numpy as np

# Data Prepration

In [8]:
dataset = load_dataset("csv", data_files="data/law_faq.csv", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
dataset[0]

{'title': '在法律中定金与订金的区别订金和定金哪个受',
 'reply': '“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担保方式，目的在于促使债务人履行债务，保障债权人的债权得以实现。签合同时，对定金必需以书面形式进行约定，同时还应约定定金的数额和交付期限。给付定金一方如果不履行债务，无权要求另一方返还定金；接受定金的一方如果不履行债务，需向另一方双倍返还债务。债务人履行债务后，依照约定，定金应抵作价款或者收回。而“订金”目前我国法律没有明确规定，它不具备定金所具有的担保性质，可视为“预付款”，当合同不能履行时，除不可抗力外，应根据双方当事人的过错承担违约责任。'}

# load model

In [14]:
from scripts.DualModel import DualModel
dual_model = DualModel.from_pretrained('./dual_model/checkpoint-315/')
dual_model = dual_model.to('cuda') 
dual_model.eval()
print('dual_model loaded')

dual_model loaded


In [15]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

# turn data (questions) into embeddings

In [25]:
from tqdm.autonotebook import tqdm
questions = dataset['title']
vectors = []
bs = 32
with torch.inference_mode():
    for i in tqdm(range(0, len(questions), bs)):
        batch_sentences = questions[i:i+bs]
        inputs = tokenizer(batch_sentences, max_length=128, padding=True, truncation=True, return_tensors="pt")
        inputs = {key: value.to(dual_model.device) for key, value in inputs.items()}
        # [1] take the pooled output
        outputs = dual_model.bert(**inputs)[1]
        vectors.append(outputs.cpu().detach().numpy())
vectors = np.concatenate(vectors, axis=0)

  0%|          | 0/570 [00:00<?, ?it/s]

In [26]:
vectors.shape

(18213, 768)

# create index

In [32]:
import faiss
dim = vectors.shape[1]
index = faiss.IndexFlatIP(dim)
faiss.normalize_L2(vectors)
index.add(vectors)
index

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x7f818aabff90> >

# question embedding

In [33]:
question = "寻衅滋事"
with torch.inference_mode():
    inputs = tokenizer(question, max_length=128, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: value.to(dual_model.device) for key, value in inputs.items()}
    vector = dual_model.bert(**inputs)[1]
    q_vector = vector.cpu().detach().numpy()
q_vector.shape

(1, 768)

# Question-Question matching (recall)

In [34]:
faiss.normalize_L2(q_vector)
scores, indices = index.search(q_vector, 5)
scores, indices

(array([[0.9037887 , 0.8666042 , 0.8658901 , 0.85802627, 0.85431874]],
       dtype=float32),
 array([[  539, 17626,  8508,  7292,  4493]]))

In [36]:
matched_questions = [questions[i] for i in indices[0]]
matched_questions

['涉嫌寻衅滋事', '元*县*管拉幅讨薪', '飞达暴力催收', '两个轻微伤够寻衅滋事', '看一下这个案子是非法拘禁还是绑']

In [38]:
matched_answers = [dataset['reply'][i] for i in indices[0]]
matched_answers

['说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行为人因日常生活中的偶发矛盾纠纷，借故生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"，但矛盾系由被害人故意引发或者被害人对矛盾激化负有主要责任的除外。行为人因婚恋、家庭、邻里、债务等纠纷，实施殴打、辱骂、恐吓他人或者损毁、占用他人财物等行为的，一般不认定为"寻衅滋事"，但经有关部门批评制止或者处理处罚后，继续实施前列行为，破坏社会秩序的除外。第二条随意殴打他人，破坏社会秩序，具有下列情形之一的，应当认定为刑法第二百九十三条第一款第一项规定的"情节恶劣":1、致一人以上轻伤或者二人以上轻微伤的;2、引起他人精神失常、自杀等严重后果的;3、多次随意殴打他人的;4、持凶器随意